## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [29]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv", keep_default_na=False)
city_data_df.drop(columns='Unnamed: 0', inplace=True)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Lebu,CL,-37.62,-73.65,57.63,68,5,13.15,clear sky
1,Zalha,RO,47.18,23.53,50.00,43,0,3.44,clear sky
2,Ifakara,TZ,-8.13,36.68,71.19,95,74,1.25,moderate rain
3,Mahebourg,MU,-20.41,57.70,78.80,78,40,11.50,scattered clouds
4,Georgetown,MY,5.41,100.34,80.01,89,40,2.48,light rain
5,Gornopravdinsk,RU,60.05,69.90,36.50,99,100,4.05,overcast clouds
6,Sitka,US,57.05,-135.33,32.00,93,90,3.44,snow
7,Vestmannaeyjar,IS,63.44,-20.27,35.60,64,75,9.22,broken clouds
8,Tuktoyaktuk,CA,69.45,-133.04,-22.00,76,75,5.75,broken clouds
9,Nikolskoye,RU,59.70,30.79,37.00,86,75,2.24,broken clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Lebu,CL,-37.62,-73.65,57.63,68,5,13.15,clear sky
1,Zalha,RO,47.18,23.53,50.00,43,0,3.44,clear sky
2,Ifakara,TZ,-8.13,36.68,71.19,95,74,1.25,moderate rain
3,Mahebourg,MU,-20.41,57.70,78.80,78,40,11.50,scattered clouds
10,Ribeira Grande,PT,38.52,-28.70,62.60,94,75,12.66,broken clouds
11,Rikitea,PF,-23.12,-134.97,78.75,71,2,9.51,clear sky
14,Vaini,TO,-21.20,-175.20,68.00,94,20,3.44,few clouds
15,Bredasdorp,ZA,-34.53,20.04,62.60,88,100,4.61,overcast clouds
16,Albany,US,42.60,-73.97,73.99,39,0,4.63,clear sky
17,Phan Thiet,VN,10.93,108.10,76.86,83,71,9.78,broken clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   324
Country                324
Lat                    324
Lng                    324
Max Temp               324
Humidity               324
Cloudiness             324
Wind Speed             324
Current Description    324
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City                   324
Country                324
Lat                    324
Lng                    324
Max Temp               324
Humidity               324
Cloudiness             324
Wind Speed             324
Current Description    324
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.NaN
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lebu,CL,57.63,clear sky,-37.62,-73.65,NaN
1,Zalha,RO,50.00,clear sky,47.18,23.53,NaN
2,Ifakara,TZ,71.19,moderate rain,-8.13,36.68,NaN
3,Mahebourg,MU,78.80,scattered clouds,-20.41,57.70,NaN
10,Ribeira Grande,PT,62.60,broken clouds,38.52,-28.70,NaN
11,Rikitea,PF,78.75,clear sky,-23.12,-134.97,NaN
14,Vaini,TO,68.00,few clouds,-21.20,-175.20,NaN
15,Bredasdorp,ZA,62.60,overcast clouds,-34.53,20.04,NaN
16,Albany,US,73.99,clear sky,42.60,-73.97,NaN
17,Phan Thiet,VN,76.86,broken clouds,10.93,108.10,NaN


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lebu,CL,57.63,clear sky,-37.62,-73.65,Hostal Las Lilas
1,Zalha,RO,50.00,clear sky,47.18,23.53,NaN
2,Ifakara,TZ,71.19,moderate rain,-8.13,36.68,Tanzanian Training Centre for International He...
3,Mahebourg,MU,78.80,scattered clouds,-20.41,57.70,Shandrani Beachcomber Resort & Spa
10,Ribeira Grande,PT,62.60,broken clouds,38.52,-28.70,Quinta da Meia Eira
11,Rikitea,PF,78.75,clear sky,-23.12,-134.97,Pension Maro'i
14,Vaini,TO,68.00,few clouds,-21.20,-175.20,Keleti Beach Resort
15,Bredasdorp,ZA,62.60,overcast clouds,-34.53,20.04,Bredasdorp Country Manor
16,Albany,US,73.99,clear sky,42.60,-73.97,NaN
17,Phan Thiet,VN,76.86,broken clouds,10.93,108.10,Hotel Saigon-PT


In [22]:
hotel_df.count()

City                   324
Country                324
Max Temp               324
Current Description    324
Lat                    324
Lng                    324
Hotel Name             309
dtype: int64

In [23]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   309
Country                309
Max Temp               309
Current Description    309
Lat                    309
Lng                    309
Hotel Name             309
dtype: int64

In [24]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lebu,CL,57.63,clear sky,-37.62,-73.65,Hostal Las Lilas
2,Ifakara,TZ,71.19,moderate rain,-8.13,36.68,Tanzanian Training Centre for International He...
3,Mahebourg,MU,78.80,scattered clouds,-20.41,57.70,Shandrani Beachcomber Resort & Spa
10,Ribeira Grande,PT,62.60,broken clouds,38.52,-28.70,Quinta da Meia Eira
11,Rikitea,PF,78.75,clear sky,-23.12,-134.97,Pension Maro'i
14,Vaini,TO,68.00,few clouds,-21.20,-175.20,Keleti Beach Resort
15,Bredasdorp,ZA,62.60,overcast clouds,-34.53,20.04,Bredasdorp Country Manor
17,Phan Thiet,VN,76.86,broken clouds,10.93,108.10,Hotel Saigon-PT
18,Ushuaia,AR,53.60,scattered clouds,-54.80,-68.30,Albatros Hotel
19,Cape Town,ZA,66.20,clear sky,-33.93,18.42,Southern Sun Cape Sun


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))